In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import random
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler, autocast

import cv2

import albumentations
from albumentations.pytorch import ToTensorV2

from sklearn.metrics import confusion_matrix, roc_auc_score, average_precision_score

from tqdm import tqdm
import argparse
import os, sys, yaml

sys.path.append('/workspace/siim-rsna-2021')
from src.logger import setup_logger, LOGGER
from src.meter import mAPMeter, AUCMeter, APMeter, AverageValueMeter
from src.utils import plot_sample_images
from src.segloss import SymmetricLovaszLoss


# import neptune.new as neptune
import wandb
import pydicom

import time
from contextlib import contextmanager

import timm

import warnings

target_columns = [
    "Negative for Pneumonia", "Typical Appearance", "Indeterminate Appearance", "Atypical Appearance", "is_none"
]

from sklearn.metrics import average_precision_score


@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')

In [2]:
df_original = pd.read_csv("/workspace/data/df_train_study_level_npy640_3_w_bbox.csv")
df = df_original.groupby('image_id').first().reset_index()

# lb best

In [3]:
exp = ["exp414_hflip", "exp416_hflip", "exp419_hflip", "exp420_hflip", "exp605_hflip", "exp520_hflip", "exp551_hflip"]

cv_all = []

print(exp)
for cv in range(5):
    pred_list = []
    pred_list.extend([np.load(f'/workspace/output/oof/{exp_name}_cv{cv}.npy') for exp_name in exp])
    pred_mean = np.mean(pred_list, axis=0)
    
    target = df[df.cv == cv][target_columns].values
    
    map_list = []
    print()
    print(f'==== cv {cv} ===')
    for i, col in enumerate(target_columns):
        map_ = average_precision_score(target[:, i], pred_mean[:, i])
        print(f"AP {col}: {map_}")
        map_list.append(map_)
    print(f"mAP: {np.mean(map_list)}")
    cv_all.append(np.mean(map_list))

print("==================================")
print(f"mAP: {np.mean(cv_all)}")
print(f"mAP: {np.std(cv_all)}")
print("==================================")

['exp414_hflip', 'exp416_hflip', 'exp419_hflip', 'exp420_hflip', 'exp605_hflip', 'exp520_hflip', 'exp551_hflip']

==== cv 0 ===
AP Negative for Pneumonia: 0.8449266431262668
AP Typical Appearance: 0.8454790486641144
AP Indeterminate Appearance: 0.33614701267225305
AP Atypical Appearance: 0.44315522519541234
AP is_none: 0.838818098318518
mAP: 0.661705205595313

==== cv 1 ===
AP Negative for Pneumonia: 0.8190509895743123
AP Typical Appearance: 0.869692799075244
AP Indeterminate Appearance: 0.3767456591899713
AP Atypical Appearance: 0.33523767835182255
AP is_none: 0.8075402025461709
mAP: 0.6416534657475041

==== cv 2 ===
AP Negative for Pneumonia: 0.843605502113421
AP Typical Appearance: 0.8592550468925507
AP Indeterminate Appearance: 0.3186060160162587
AP Atypical Appearance: 0.36448903435795554
AP is_none: 0.8372955467549843
mAP: 0.644650229227034

==== cv 3 ===
AP Negative for Pneumonia: 0.8257041636117594
AP Typical Appearance: 0.84889188927688
AP Indeterminate Appearance: 0.331872988

# all

In [5]:
exp = ["exp414_hflip", "exp415_hflip", "exp416_hflip", "exp417_hflip",
       "exp419_hflip", "exp420_hflip", "exp605_hflip", "exp604_hflip", "exp520_hflip",
       "exp551_hflip", "exp552_hflip", "exp553_hflip"]

cv_all = []

print(exp)
for cv in range(5):
    pred_list = []
    pred_list.extend([np.load(f'/workspace/output/oof/{exp_name}_cv{cv}.npy') for exp_name in exp])
    pred_mean = np.mean(pred_list, axis=0)
    
    target = df[df.cv == cv][target_columns].values
    
    map_list = []
    print()
    print(f'==== cv {cv} ===')
    for i, col in enumerate(target_columns):
        map_ = average_precision_score(target[:, i], pred_mean[:, i])
        print(f"AP {col}: {map_}")
        map_list.append(map_)
    print(f"mAP: {np.mean(map_list)}")
    cv_all.append(np.mean(map_list))

print("==================================")
print(f"mAP: {np.mean(cv_all)}")
print(f"mAP: {np.std(cv_all)}")
print("==================================")

['exp414_hflip', 'exp415_hflip', 'exp416_hflip', 'exp417_hflip', 'exp419_hflip', 'exp420_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']

==== cv 0 ===
AP Negative for Pneumonia: 0.8491098924380744
AP Typical Appearance: 0.8476076167714446
AP Indeterminate Appearance: 0.33343118219383827
AP Atypical Appearance: 0.4500605094334274
AP is_none: 0.8414700204938704
mAP: 0.664335844266131

==== cv 1 ===
AP Negative for Pneumonia: 0.8190622546434813
AP Typical Appearance: 0.8692715872166321
AP Indeterminate Appearance: 0.3697151560650166
AP Atypical Appearance: 0.3434848866481594
AP is_none: 0.8070601575698482
mAP: 0.6417188084286275

==== cv 2 ===
AP Negative for Pneumonia: 0.8415974507390225
AP Typical Appearance: 0.8589504030530617
AP Indeterminate Appearance: 0.3217563508404653
AP Atypical Appearance: 0.3645626555005531
AP is_none: 0.8360537923658935
mAP: 0.6445841304997992

==== cv 3 ===
AP Negative for Pneumonia: 0.8279069872079704

In [8]:
exp = ["exp414_hflip", "exp415_hflip", "exp416_hflip", "exp417_hflip",
       "exp419_hflip", "exp420_hflip", "exp605_hflip", "exp604_hflip", "exp520_hflip",
       "exp551_hflip", "exp552_hflip", "exp553_hflip"]

for i in range(len(exp)):
    exp = ["exp414_hflip", "exp415_hflip", "exp416_hflip", "exp417_hflip",
       "exp419_hflip", "exp420_hflip", "exp605_hflip", "exp604_hflip", "exp520_hflip",
       "exp551_hflip", "exp552_hflip", "exp553_hflip"]

    print(exp.pop(i))

    cv_all = []

    print(exp)
    for cv in range(5):
        pred_list = []
        pred_list.extend([np.load(f'/workspace/output/oof/{exp_name}_cv{cv}.npy') for exp_name in exp])
        pred_mean = np.mean(pred_list, axis=0)

        target = df[df.cv == cv][target_columns].values

        map_list = []
#         print()
#         print(f'==== cv {cv} ===')
        for i, col in enumerate(target_columns):
            map_ = average_precision_score(target[:, i], pred_mean[:, i])
#             print(f"AP {col}: {map_}")
            map_list.append(map_)
#         print(f"mAP: {np.mean(map_list)}")
        cv_all.append(np.mean(map_list))

    print("==================================")
    print(f"mAP: {np.mean(cv_all)}")
    print(f"mAP: {np.std(cv_all)}")
    print("==================================")

exp414_hflip
['exp415_hflip', 'exp416_hflip', 'exp417_hflip', 'exp419_hflip', 'exp420_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.6394409059817695
mAP: 0.020370139452893597
exp415_hflip
['exp414_hflip', 'exp416_hflip', 'exp417_hflip', 'exp419_hflip', 'exp420_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.6397515126733017
mAP: 0.020500432624747825
exp416_hflip
['exp414_hflip', 'exp415_hflip', 'exp417_hflip', 'exp419_hflip', 'exp420_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.6401952246315108
mAP: 0.020574947119358183
exp417_hflip
['exp414_hflip', 'exp415_hflip', 'exp416_hflip', 'exp419_hflip', 'exp420_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.6396880390740054
mAP: 0.020938592705169307
exp419_hflip
['exp414_hflip', 'exp415_hf

In [9]:
drop = ["exp420_hflip"]

In [10]:
exp = ["exp414_hflip", "exp415_hflip", "exp416_hflip", "exp417_hflip",
       "exp419_hflip", "exp605_hflip", "exp604_hflip", "exp520_hflip",
       "exp551_hflip", "exp552_hflip", "exp553_hflip"]

for i in range(len(exp)):
    exp = ["exp414_hflip", "exp415_hflip", "exp416_hflip", "exp417_hflip",
       "exp419_hflip", "exp605_hflip", "exp604_hflip", "exp520_hflip",
       "exp551_hflip", "exp552_hflip", "exp553_hflip"]

    print(exp.pop(i))

    cv_all = []

    print(exp)
    for cv in range(5):
        pred_list = []
        pred_list.extend([np.load(f'/workspace/output/oof/{exp_name}_cv{cv}.npy') for exp_name in exp])
        pred_mean = np.mean(pred_list, axis=0)

        target = df[df.cv == cv][target_columns].values

        map_list = []
#         print()
#         print(f'==== cv {cv} ===')
        for i, col in enumerate(target_columns):
            map_ = average_precision_score(target[:, i], pred_mean[:, i])
#             print(f"AP {col}: {map_}")
            map_list.append(map_)
#         print(f"mAP: {np.mean(map_list)}")
        cv_all.append(np.mean(map_list))

    print("==================================")
    print(f"mAP: {np.mean(cv_all)}")
    print(f"mAP: {np.std(cv_all)}")
    print("==================================")

exp414_hflip
['exp415_hflip', 'exp416_hflip', 'exp417_hflip', 'exp419_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.6400499094788282
mAP: 0.019986487593841095
exp415_hflip
['exp414_hflip', 'exp416_hflip', 'exp417_hflip', 'exp419_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.6404231583158243
mAP: 0.020146446374310376
exp416_hflip
['exp414_hflip', 'exp415_hflip', 'exp417_hflip', 'exp419_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.6408222282018153
mAP: 0.02014456890350493
exp417_hflip
['exp414_hflip', 'exp415_hflip', 'exp416_hflip', 'exp419_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.6399969430479081
mAP: 0.020812227998550915
exp419_hflip
['exp414_hflip', 'exp415_hflip', 'exp416_hflip', 'exp417_hflip', 'exp605_hflip', 'exp604_hfl

In [11]:
drop += ["exp416_hflip"]

In [14]:
exp = ["exp414_hflip", "exp415_hflip", "exp417_hflip",
       "exp419_hflip", "exp605_hflip", "exp604_hflip", "exp520_hflip",
       "exp551_hflip", "exp552_hflip", "exp553_hflip"]

for i in range(len(exp)):
    exp = ["exp414_hflip", "exp415_hflip", "exp417_hflip",
       "exp419_hflip", "exp605_hflip", "exp604_hflip", "exp520_hflip",
       "exp551_hflip", "exp552_hflip", "exp553_hflip"]

    print(exp.pop(i))

    cv_all = []

    print(exp)
    for cv in range(5):
        pred_list = []
        pred_list.extend([np.load(f'/workspace/output/oof/{exp_name}_cv{cv}.npy') for exp_name in exp])
        pred_mean = np.mean(pred_list, axis=0)

        target = df[df.cv == cv][target_columns].values

        map_list = []
#         print()
#         print(f'==== cv {cv} ===')
        for i, col in enumerate(target_columns):
            map_ = average_precision_score(target[:, i], pred_mean[:, i])
#             print(f"AP {col}: {map_}")
            map_list.append(map_)
#         print(f"mAP: {np.mean(map_list)}")
        cv_all.append(np.mean(map_list))

    print("==================================")
    print(f"mAP: {np.mean(cv_all)}")
    print(f"mAP: {np.std(cv_all)}")
    print("==================================")

exp414_hflip
['exp415_hflip', 'exp417_hflip', 'exp419_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.6404796915313883
mAP: 0.020069801589593775
exp415_hflip
['exp414_hflip', 'exp417_hflip', 'exp419_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.6412180130897843
mAP: 0.020316817889682627
exp417_hflip
['exp414_hflip', 'exp415_hflip', 'exp419_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.6399339667875485
mAP: 0.020956390070836634
exp419_hflip
['exp414_hflip', 'exp415_hflip', 'exp417_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.6408573734453208
mAP: 0.019917880028622034
exp605_hflip
['exp414_hflip', 'exp415_hflip', 'exp417_hflip', 'exp419_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.639

In [13]:
drop += ["exp415_hflip"]

In [15]:
exp = ["exp414_hflip", "exp417_hflip",
       "exp419_hflip", "exp605_hflip", "exp604_hflip", "exp520_hflip",
       "exp551_hflip", "exp552_hflip", "exp553_hflip"]

for i in range(len(exp)):
    exp = ["exp414_hflip", "exp417_hflip",
       "exp419_hflip", "exp605_hflip", "exp604_hflip", "exp520_hflip",
       "exp551_hflip", "exp552_hflip", "exp553_hflip"]

    print(exp.pop(i))

    cv_all = []

    print(exp)
    for cv in range(5):
        pred_list = []
        pred_list.extend([np.load(f'/workspace/output/oof/{exp_name}_cv{cv}.npy') for exp_name in exp])
        pred_mean = np.mean(pred_list, axis=0)

        target = df[df.cv == cv][target_columns].values

        map_list = []
#         print()
#         print(f'==== cv {cv} ===')
        for i, col in enumerate(target_columns):
            map_ = average_precision_score(target[:, i], pred_mean[:, i])
#             print(f"AP {col}: {map_}")
            map_list.append(map_)
#         print(f"mAP: {np.mean(map_list)}")
        cv_all.append(np.mean(map_list))

    print("==================================")
    print(f"mAP: {np.mean(cv_all)}")
    print(f"mAP: {np.std(cv_all)}")
    print("==================================")

exp414_hflip
['exp417_hflip', 'exp419_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.64051091859393
mAP: 0.019728045377635273
exp417_hflip
['exp414_hflip', 'exp419_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.6403351472486607
mAP: 0.020677312582168043
exp419_hflip
['exp414_hflip', 'exp417_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.6414239734247122
mAP: 0.019591032039346763
exp605_hflip
['exp414_hflip', 'exp417_hflip', 'exp419_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.639669811195908
mAP: 0.021463684952328577
exp604_hflip
['exp414_hflip', 'exp417_hflip', 'exp419_hflip', 'exp605_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.6403716204654404
mAP: 0.02099650155861804
exp520_hflip
['exp414_hflip', 'exp417_hflip'

In [16]:
drop += ["exp419_hflip"]

# コレがベスト

In [22]:
exp = ["exp414_hflip", "exp417_hflip",
       "exp605_hflip", "exp604_hflip", "exp520_hflip",
       "exp551_hflip", "exp552_hflip", "exp553_hflip"]

In [17]:
exp = ["exp414_hflip", "exp417_hflip",
       "exp605_hflip", "exp604_hflip", "exp520_hflip",
       "exp551_hflip", "exp552_hflip", "exp553_hflip"]

for i in range(len(exp)):
    exp = ["exp414_hflip", "exp417_hflip",
       "exp605_hflip", "exp604_hflip", "exp520_hflip",
       "exp551_hflip", "exp552_hflip", "exp553_hflip"]

    print(exp.pop(i))

    cv_all = []

    print(exp)
    for cv in range(5):
        pred_list = []
        pred_list.extend([np.load(f'/workspace/output/oof/{exp_name}_cv{cv}.npy') for exp_name in exp])
        pred_mean = np.mean(pred_list, axis=0)

        target = df[df.cv == cv][target_columns].values

        map_list = []
#         print()
#         print(f'==== cv {cv} ===')
        for i, col in enumerate(target_columns):
            map_ = average_precision_score(target[:, i], pred_mean[:, i])
#             print(f"AP {col}: {map_}")
            map_list.append(map_)
#         print(f"mAP: {np.mean(map_list)}")
        cv_all.append(np.mean(map_list))

    print("==================================")
    print(f"mAP: {np.mean(cv_all)}")
    print(f"mAP: {np.std(cv_all)}")
    print("==================================")

exp414_hflip
['exp417_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.640903172856855
mAP: 0.019044822711293874
exp417_hflip
['exp414_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.640362001971283
mAP: 0.019759765492104748
exp605_hflip
['exp414_hflip', 'exp417_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.6397810649600721
mAP: 0.020567691454882405
exp604_hflip
['exp414_hflip', 'exp417_hflip', 'exp605_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.6407377720859384
mAP: 0.02054974560877108
exp520_hflip
['exp414_hflip', 'exp417_hflip', 'exp605_hflip', 'exp604_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.6410113345912125
mAP: 0.0195329585946276
exp551_hflip
['exp414_hflip', 'exp417_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp552_hflip', 'exp553_hflip']

In [20]:
drop

['exp420_hflip', 'exp416_hflip', 'exp415_hflip', 'exp419_hflip']

# dropしたものを1個づつ足していく

In [21]:
exp = ["exp414_hflip", "exp417_hflip",
       "exp605_hflip", "exp604_hflip", "exp520_hflip",
       "exp551_hflip", "exp552_hflip", "exp553_hflip"]

for i in range(len(drop)):
    exp = ["exp414_hflip", "exp417_hflip",
       "exp605_hflip", "exp604_hflip", "exp520_hflip",
       "exp551_hflip", "exp552_hflip", "exp553_hflip"]

    exp += [drop[i]]

    cv_all = []

    print(exp)
    for cv in range(5):
        pred_list = []
        pred_list.extend([np.load(f'/workspace/output/oof/{exp_name}_cv{cv}.npy') for exp_name in exp])
        pred_mean = np.mean(pred_list, axis=0)

        target = df[df.cv == cv][target_columns].values

        map_list = []
#         print()
#         print(f'==== cv {cv} ===')
        for i, col in enumerate(target_columns):
            map_ = average_precision_score(target[:, i], pred_mean[:, i])
#             print(f"AP {col}: {map_}")
            map_list.append(map_)
#         print(f"mAP: {np.mean(map_list)}")
        cv_all.append(np.mean(map_list))

    print("==================================")
    print(f"mAP: {np.mean(cv_all)}")
    print(f"mAP: {np.std(cv_all)}")
    print("==================================")

['exp414_hflip', 'exp417_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip', 'exp420_hflip']
mAP: 0.640760226709953
mAP: 0.020249743439300867
['exp414_hflip', 'exp417_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip', 'exp416_hflip']
mAP: 0.640795046684427
mAP: 0.019855745475750943
['exp414_hflip', 'exp417_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip', 'exp415_hflip']
mAP: 0.6408573734453208
mAP: 0.019917880028622034
['exp414_hflip', 'exp417_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip', 'exp419_hflip']
mAP: 0.6412179642694781
mAP: 0.02031676004553769


# Best再掲

In [33]:
    exp = ["exp414_hflip", "exp417_hflip",
       "exp605_hflip", "exp604_hflip", "exp520_hflip",
       "exp551_hflip", "exp552_hflip", "exp553_hflip"]

    cv_all = []

    print(exp)
    for cv in range(5):
        pred_list = []
        pred_list.extend([np.load(f'/workspace/output/oof/{exp_name}_cv{cv}.npy') for exp_name in exp])
        pred_mean = np.mean(pred_list, axis=0)

        target = df[df.cv == cv][target_columns].values

        map_list = []
        print()
        print(f'==== cv {cv} ===')
        for i, col in enumerate(target_columns):
            map_ = average_precision_score(target[:, i], pred_mean[:, i])
            print(f"AP {col}: {map_}")
            map_list.append(map_)
        print(f"mAP: {np.mean(map_list)}")
        cv_all.append(np.mean(map_list))

    print("==================================")
    print(f"mAP: {np.mean(cv_all)}")
    print(f"mAP: {np.std(cv_all)}")
    print("==================================")

['exp414_hflip', 'exp417_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']

==== cv 0 ===
AP Negative for Pneumonia: 0.8494583886062388
AP Typical Appearance: 0.848991618365087
AP Indeterminate Appearance: 0.3318601399160621
AP Atypical Appearance: 0.4556530301656698
AP is_none: 0.840150427484673
mAP: 0.6652227209075461

==== cv 1 ===
AP Negative for Pneumonia: 0.8194298615567489
AP Typical Appearance: 0.8683637207779785
AP Indeterminate Appearance: 0.3699603007298199
AP Atypical Appearance: 0.3489032893106502
AP is_none: 0.8069964588268366
mAP: 0.6427307262404067

==== cv 2 ===
AP Negative for Pneumonia: 0.8414470957057392
AP Typical Appearance: 0.8585156927836197
AP Indeterminate Appearance: 0.3290846899808617
AP Atypical Appearance: 0.3658011603545256
AP is_none: 0.835960715226159
mAP: 0.646161870810181

==== cv 3 ===
AP Negative for Pneumonia: 0.8292131941068765
AP Typical Appearance: 0.8502162830724579
AP Indeterminate Appearan

# 他、てきとーに試した

In [27]:
exp = ["exp414_hflip", "exp417_hflip", "exp605_hflip", "exp604_hflip", "exp520_hflip", "exp551_hflip", "exp552_hflip", "exp553_hflip"]

cv_all = []

print(exp)
for cv in range(5):
    pred_list = []
    pred_list.extend([np.load(f'/workspace/output/oof/{exp_name}_cv{cv}.npy') for exp_name in exp])
    pred_mean = np.mean(pred_list, axis=0)

    target = df[df.cv == cv][target_columns].values

    map_list = []
#         print()
#         print(f'==== cv {cv} ===')
    for i, col in enumerate(target_columns):
        map_ = average_precision_score(target[:, i], pred_mean[:, i])
#             print(f"AP {col}: {map_}")
        map_list.append(map_)
#         print(f"mAP: {np.mean(map_list)}")
    cv_all.append(np.mean(map_list))

print("==================================")
print(f"mAP: {np.mean(cv_all)}")
print(f"mAP: {np.std(cv_all)}")
print("==================================")

['exp414_hflip', 'exp417_hflip', 'exp605_hflip', 'exp604_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.6414239734247122
mAP: 0.019591032039346763


In [29]:
exp = ["exp520_hflip", "exp551_hflip", "exp552_hflip", "exp553_hflip"]

cv_all = []

print(exp)
for cv in range(5):
    pred_list = []
    pred_list.extend([np.load(f'/workspace/output/oof/{exp_name}_cv{cv}.npy') for exp_name in exp])
    pred_mean = np.mean(pred_list, axis=0)

    target = df[df.cv == cv][target_columns].values

    map_list = []
#         print()
#         print(f'==== cv {cv} ===')
    for i, col in enumerate(target_columns):
        map_ = average_precision_score(target[:, i], pred_mean[:, i])
#             print(f"AP {col}: {map_}")
        map_list.append(map_)
#         print(f"mAP: {np.mean(map_list)}")
    cv_all.append(np.mean(map_list))

print("==================================")
print(f"mAP: {np.mean(cv_all)}")
print(f"mAP: {np.std(cv_all)}")
print("==================================")

['exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.6361170269950776
mAP: 0.022111422475202598


In [31]:
exp = ["exp414_hflip", "exp415_hflip", "exp416_hflip", "exp417_hflip"]

cv_all = []

print(exp)
for cv in range(5):
    pred_list = []
    pred_list.extend([np.load(f'/workspace/output/oof/{exp_name}_cv{cv}.npy') for exp_name in exp])
    pred_mean = np.mean(pred_list, axis=0)

    target = df[df.cv == cv][target_columns].values

    map_list = []
#         print()
#         print(f'==== cv {cv} ===')
    for i, col in enumerate(target_columns):
        map_ = average_precision_score(target[:, i], pred_mean[:, i])
#             print(f"AP {col}: {map_}")
        map_list.append(map_)
#         print(f"mAP: {np.mean(map_list)}")
    cv_all.append(np.mean(map_list))

print("==================================")
print(f"mAP: {np.mean(cv_all)}")
print(f"mAP: {np.std(cv_all)}")
print("==================================")

['exp414_hflip', 'exp415_hflip', 'exp416_hflip', 'exp417_hflip']
mAP: 0.6359726395337224
mAP: 0.0224561904167853


In [32]:
exp = ["exp414_hflip", "exp415_hflip", "exp416_hflip", "exp417_hflip", "exp520_hflip", "exp551_hflip", "exp552_hflip", "exp553_hflip"]

cv_all = []

print(exp)
for cv in range(5):
    pred_list = []
    pred_list.extend([np.load(f'/workspace/output/oof/{exp_name}_cv{cv}.npy') for exp_name in exp])
    pred_mean = np.mean(pred_list, axis=0)

    target = df[df.cv == cv][target_columns].values

    map_list = []
#         print()
#         print(f'==== cv {cv} ===')
    for i, col in enumerate(target_columns):
        map_ = average_precision_score(target[:, i], pred_mean[:, i])
#             print(f"AP {col}: {map_}")
        map_list.append(map_)
#         print(f"mAP: {np.mean(map_list)}")
    cv_all.append(np.mean(map_list))

print("==================================")
print(f"mAP: {np.mean(cv_all)}")
print(f"mAP: {np.std(cv_all)}")
print("==================================")

['exp414_hflip', 'exp415_hflip', 'exp416_hflip', 'exp417_hflip', 'exp520_hflip', 'exp551_hflip', 'exp552_hflip', 'exp553_hflip']
mAP: 0.6377320466456449
mAP: 0.022100108368689052


In [35]:
for exp in ["exp520_hflip", "exp551_hflip", "exp552_hflip", "exp553_hflip"]:

    cv_all = []

    print(exp)
    for cv in range(5):
        pred_list = []
        pred_list.extend([np.load(f'/workspace/output/oof/{exp_name}_cv{cv}.npy') for exp_name in [exp]])
        pred_mean = np.mean(pred_list, axis=0)

        target = df[df.cv == cv][target_columns].values

        map_list = []
    #         print()
    #         print(f'==== cv {cv} ===')
        for i, col in enumerate(target_columns):
            map_ = average_precision_score(target[:, i], pred_mean[:, i])
    #             print(f"AP {col}: {map_}")
            map_list.append(map_)
    #         print(f"mAP: {np.mean(map_list)}")
        cv_all.append(np.mean(map_list))

    print("==================================")
    print(f"mAP: {np.mean(cv_all)}")
    print(f"mAP: {np.std(cv_all)}")
    print("==================================")

exp520_hflip
mAP: 0.6287490280101553
mAP: 0.01931015518746067
exp551_hflip
mAP: 0.6279055055844953
mAP: 0.020731717361017175
exp552_hflip
mAP: 0.6221740055398752
mAP: 0.023425037482202656
exp553_hflip
mAP: 0.6281554880812512
mAP: 0.02060292285288922


In [38]:
for exp in ["exp551_hflip"]:

    cv_all = []

    print(exp)
    for cv in range(5):
        pred_list = []
        pred_list.extend([np.load(f'/workspace/output/oof/{exp_name}_cv{cv}.npy') for exp_name in [exp]])
        pred_mean = np.mean(pred_list, axis=0)

        target = df[df.cv == cv][target_columns].values

        map_list = []
    #         print()
    #         print(f'==== cv {cv} ===')
        for i, col in enumerate(target_columns):
            map_ = average_precision_score(target[:, i], pred_mean[:, i])
    #             print(f"AP {col}: {map_}")
            map_list.append(map_)
    #         print(f"mAP: {np.mean(map_list)}")
        cv_all.append(np.mean(map_list))

    print("==================================")
    print(f"mAP: {np.mean(cv_all)}")
    print(f"mAP: {np.std(cv_all)}")
    print("==================================")
    
    print("==================================")
    print("= Study level")
    print(f"mAP: {np.mean(cv_all[:4])}")
    print(f"mAP: {np.std(cv_all[:4])}")
    print("==================================")

exp551_hflip
mAP: 0.6279055055844953
mAP: 0.020731717361017175
= Study level
mAP: 0.6372154258404286
mAP: 0.010192254660387524


In [37]:
0.6279055055844953 * 5 / 6

0.5232545879870795

In [39]:
0.6372154258404286 * 4 / 6

0.424810283893619

In [40]:
(0.5509 + 0.541 + 0.537 + 0.538 + 0.525) / 5

0.53838

In [41]:
0.53838 / 6

0.08972999999999999